In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time

import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
import pickle

import pandas as pd
import re
from pandas.io.json import json_normalize
import json

In [3]:
def convert_ids(ids_in_csv):
    return pd.to_numeric(ids_in_csv, errors='coerce').astype('int64')

def convert_to_float(ids_in_csv):
    return pd.to_numeric(ids_in_csv, errors='coerce').astype('float64')

def to_json(csv_entry):
    return json.loads(re.sub('\'', '"', csv_entry))

movies_metadata_df = pd.read_csv('the-movies-dataset/movies_metadata.csv'
                                 , converters={ 'id': lambda x: convert_ids(x)
                                               , 'imdb_id': lambda x: convert_ids(x)
                                               ,'popularity': lambda x: convert_to_float(x)
                                               ,'genres': lambda x: to_json(x)}
                                 , usecols=['id', 'original_title'
                                                , 'genres', 'homepage'
                                                , 'overview', 'popularity', 'poster_path'
                                                , 'release_date', 'revenue', 'runtime'
                                                , 'spoken_languages', 'tagline', 'title'
                                                , 'vote_average', 'vote_count']
                                , dtype={'populariy': np.float64}
                                , parse_dates=True)

In [4]:
mini_df = movies_metadata_df[['id', 'popularity', 'release_date', 'revenue', 'runtime', 'vote_average', 'genres']]

In [5]:
mini_df['release_year'] = pd.to_datetime(movies_metadata_df['release_date'], errors='coerce').apply(lambda x: x.year)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
mini_df.head()

,id,popularity,release_date,revenue,runtime,vote_average,genres,release_year
0,862,21.946943,1995-10-30,373554033.0,81.0,7.7,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995.0
1,8844,17.015539,1995-12-15,262797249.0,104.0,6.9,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995.0
2,15602,11.712900,1995-12-22,0.0,101.0,6.5,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995.0
3,31357,3.859495,1995-12-22,81452156.0,127.0,6.1,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995.0
4,11862,8.387519,1995-02-10,76578911.0,106.0,5.7,"[{'id': 35, 'name': 'Comedy'}]",1995.0


In [7]:
[list(item.values())[0] for item in mini_df[['genres']].loc[0][0]]

[16, 35, 10751]

In [8]:
json_normalize(mini_df[['genres']].loc[0][0])

,id,name
0,16,Animation
1,35,Comedy
2,10751,Family


In [9]:
json_normalize(mini_df[['genres']].loc[1][0])

,id,name
0,12,Adventure
1,14,Fantasy
2,10751,Family


In [10]:
json_normalize(mini_df[['genres']].loc[2][0])

,id,name
0,10749,Romance
1,35,Comedy


In [13]:
pd.get_dummies(mini_df['genres'])

TypeError: unhashable type: 'list'

In [12]:
mini_df['genre_ids'] = mini_df['genres'].apply(lambda x: [list(item.values())[0] for item in x])

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
for i, lst in enumerate(mini_df.genre_ids):
    for j, item in enumerate(lst):
        if type(item) == str:
            print(f'{type(item)} found at series index{i}, array index{j}')

<class 'str'> found at series index19730, array index0
<class 'str'> found at series index19730, array index1
<class 'str'> found at series index19730, array index2
<class 'str'> found at series index29503, array index0
<class 'str'> found at series index29503, array index1
<class 'str'> found at series index29503, array index2
<class 'str'> found at series index29503, array index3
<class 'str'> found at series index29503, array index4
<class 'str'> found at series index35587, array index0
<class 'str'> found at series index35587, array index1
<class 'str'> found at series index35587, array index2
<class 'str'> found at series index35587, array index3


In [15]:
mini_df_dropped_na = mini_df.dropna()

In [16]:
mini_df_dropped_na.shape

(45130, 9)

In [17]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
encoded_categories = mlb.fit_transform(mini_df_dropped_na['genre_ids'])

In [18]:
encoded_categories_df = pd.DataFrame(encoded_categories)

In [19]:
encoded_categories_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
7,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [20]:
content_filter_df = pd.concat([mini_df_dropped_na, encoded_categories_df], axis=1)

In [21]:
content_filter_df.to_pickle('content_filter_df.pkl')

In [22]:
content_filter_df

,id,popularity,release_date,revenue,runtime,vote_average,genres,release_year,genre_ids,0,...,10,11,12,13,14,15,16,17,18,19
0,862.0,21.946943,1995-10-30,373554033.0,81.0,7.7,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995.0,"[16, 35, 10751]",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,8844.0,17.015539,1995-12-15,262797249.0,104.0,6.9,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995.0,"[12, 14, 10751]",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,15602.0,11.712900,1995-12-22,0.0,101.0,6.5,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",1995.0,"[10749, 35]",0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,31357.0,3.859495,1995-12-22,81452156.0,127.0,6.1,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995.0,"[35, 18, 10749]",0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,11862.0,8.387519,1995-02-10,76578911.0,106.0,5.7,"[{'id': 35, 'name': 'Comedy'}]",1995.0,[35],0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,949.0,17.924927,1995-12-15,187436818.0,170.0,7.7,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",1995.0,"[28, 80, 18, 53]",0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,11860.0,6.677277,1995-12-15,0.0,127.0,6.2,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",1995.0,"[35, 10749]",0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,45325.0,2.561161,1995-12-22,0.0,97.0,5.4,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",1995.0,"[28, 12, 18, 10751]",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8,9091.0,5.231580,1995-12-22,64350171.0,106.0,5.5,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",1995.0,"[28, 12, 53]",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,710.0,14.686036,1995-11-16,352194034.0,130.0,6.6,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",1995.0,"[12, 28, 53]",1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
